In [3]:
import requests
from collections import Counter
import re
import csv
from datetime import datetime, timedelta
from tqdm import tqdm

# Step 1: Search for the Word in PubMed and Retrieve Publication PMIDs
def search_pubmed(word, max_results=100):
    print(f"Searching for the word '{word}' in PubMed...")

    esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    
    # Calculate the date range for the last 3 years
    end_date = datetime.now().strftime("%Y/%m/%d")
    start_date = (datetime.now() - timedelta(days=3*365)).strftime("%Y/%m/%d")

    params = {
        'db': 'pubmed',
        'term': word,
        'retmax': max_results,
        'retmode': 'json',
        'mindate': start_date,
        'maxdate': end_date
    }

    response = requests.get(esearch_url, params=params)
    
    if response.status_code != 200:
        print("Error fetching data from PubMed API.")
        return []
    
    pmids = response.json().get('esearchresult', {}).get('idlist', [])
    
    print(f"Found {len(pmids)} publications with the word '{word}'.")
    print(f"PMIDs: {pmids}\n")
    
    return pmids

# Step 2: Fetch Publication Details and Extract Keywords
def fetch_keywords(pmids):
    if not pmids:
        return [], []
    
    print(f"Fetching keywords for PMIDs: {pmids}")
    
    efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
    params = {
        'db': 'pubmed',
        'id': ','.join(pmids),
        'retmode': 'xml'
    }
    response = requests.get(efetch_url, params=params)
    
    if response.status_code != 200:
        print("Error fetching abstracts and keywords from PubMed.")
        return [], []
    
    xml_text = response.text
    
    # Extract keywords from the XML response
    keywords = re.findall(r'<Keyword MajorTopicYN="N">(.*?)</Keyword>', xml_text)

    # Extract links to the articles
    article_links = [f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/" for pmid in pmids]
    
    print(f"Extracted Keywords: {keywords}")
    print(f"Article Links: {article_links}\n")
    
    return keywords, article_links

# Step 3: Analyze Intersections of Keywords
def find_top_terms(keywords, top_n=5):
    print(f"Analyzing {len(keywords)} keywords to find the top {top_n}...\n")
    
    if not keywords:
        print("No keywords found in the publications.")
        return []
    
    keyword_counter = Counter(keywords)
    common_keywords = keyword_counter.most_common(top_n)
    
    print(f"Top {top_n} keywords:")
    for keyword, count in common_keywords:
        print(f"{keyword}: {count}")
    print("\n")
    
    return common_keywords

# Step 4: Main Function to Execute the Analysis and Save Results
def main():
    input_csv = '../data/semantic_network_umls_terms_population_group.csv'
    output_csv = '../data/semantic_network_umls_terms_final_with_keywords.csv'

    # Step 1: Read the output CSV to get already processed concepts
    processed_concepts = set()
    try:
        with open(output_csv, mode='r') as outfile:
            reader = csv.DictReader(outfile)
            processed_concepts = {row['STR'] for row in reader}
    except FileNotFoundError:
        print("Output file not found. Starting from scratch.")

    with open(input_csv, mode='r') as infile:
        reader = list(csv.DictReader(infile))
        fieldnames = reader[0].keys() | {'Top_Keywords_and_Counts', 'Article_Links'}

        rows_to_save = []
        counter = 0  # Counter to track processed rows

        # Use tqdm to track progress
        for row in tqdm(reader, desc="Processing concepts"):
            concept = row['STR']
            if concept in processed_concepts:
                continue

            print(f"Processing concept: {concept}")

            # Step 1: Search for the concept in PubMed with date filtering
            pmids = search_pubmed(concept, max_results=200)
            
            if not pmids:
                row['Top_Keywords_and_Counts'] = ''
                row['Article_Links'] = ''
                rows_to_save.append(row)
                continue
            
            # Step 2: Extract keywords and article links
            all_keywords = []
            all_links = []
            for pmid_batch in [pmids[i:i + 10] for i in range(0, len(pmids), 10)]:  # Batch processing
                keywords, links = fetch_keywords(pmid_batch)
                if keywords:
                    all_keywords.extend(keywords)
                if links:
                    all_links.extend(links)
            
            if not all_keywords:
                row['Top_Keywords_and_Counts'] = ''
                row['Article_Links'] = ''
                rows_to_save.append(row)
                continue
            
            # Step 3: Find and save the top 5 keywords
            top_keywords = find_top_terms(all_keywords, top_n=5)
            keywords_str = '; '.join([f"{keyword}: {count}" for keyword, count in top_keywords])
            links_str = ', '.join(all_links)
            
            # Save the results temporarily
            row['Top_Keywords_and_Counts'] = keywords_str
            row['Article_Links'] = links_str
            rows_to_save.append(row)

            # Increment counter and check if we need to save
            counter += 1
            if counter % 25 == 0:
                with open(output_csv, mode='a', newline='') as outfile:
                    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
                    if outfile.tell() == 0:
                        writer.writeheader()
                    writer.writerows(rows_to_save)
                rows_to_save = []  # Clear saved rows

        # Save any remaining rows
        if rows_to_save:
            with open(output_csv, mode='a', newline='') as outfile:
                writer = csv.DictWriter(outfile, fieldnames=fieldnames)
                if outfile.tell() == 0:
                    writer.writeheader()
                writer.writerows(rows_to_save)

if __name__ == "__main__":
    main()

Output file not found. Starting from scratch.


Processing concepts:   0%|                              | 0/663 [00:00<?, ?it/s]

Processing concept: acceptors
Searching for the word 'acceptors' in PubMed...
Found 200 publications with the word 'acceptors'.
PMIDs: ['39738147', '39738040', '39737871', '39737706', '39737004', '39736493', '39736261', '39736187', '39735554', '39734058', '39733143', '39729988', '39729546', '39728484', '39728302', '39727878', '39727869', '39727308', '39727299', '39725658', '39725606', '39725505', '39725141', '39724707', '39723892', '39723834', '39723738', '39723552', '39722888', '39722789', '39722788', '39722787', '39721993', '39721219', '39720129', '39719364', '39718297', '39718269', '39718105', '39717882', '39717821', '39717611', '39716945', '39716857', '39715736', '39715583', '39715025', '39715009', '39714948', '39714861', '39714854', '39714586', '39714555', '39714412', '39714349', '39714346', '39714332', '39714328', '39714209', '39714131', '39713659', '39713657', '39713457', '39712169', '39712147', '39711329', '39711261', '39711252', '39711011', '39708766', '39707995', '39707764', 

Extracted Keywords: ['Fluorescence imaging', 'NIR-II fluorophore', 'inhibiting H-aggregates', 'nanofluorophores', 'Donor-acceptor polymers', 'Lithium metal batteries', 'Separator modification', 'Solvation Structure', 'Triazine', 'Fluorescent probe', 'Human serum', 'Hydrogen sulfide', 'Naphthalimide', 'Catalysis', 'Chemical Bonding', 'Intermediates', 'Organometallic Chemistry', 'Spectroscopy', 'exciplexes', 'long lifetime', 'organic afterglow materials', 'polymers', 'room-temperature phosphorescence', 'Additive', 'Isomerization Strategy', 'Molecular Aggregation', 'Organic Solar Cells', 'gangliosides', 'glycosylation', 'glycosyltransferase', 'oligosaccharides', 'Juncus', 'Spartina/Sporobolus', 'metagenome-assembled genomes', 'metagenomic and metatranscriptomic sequencing', 'microbial ecology', 'salt marsh microbial ecology']
Article Links: ['https://pubmed.ncbi.nlm.nih.gov/39714854/', 'https://pubmed.ncbi.nlm.nih.gov/39714586/', 'https://pubmed.ncbi.nlm.nih.gov/39714555/', 'https://pubme

Extracted Keywords: ['halogen bond', 'ion pair', 'porous frameworks', 'supramolecular chemistry', 'Porphyrin', 'binaphthyl', 'donor-acceptor interaction', 'helical supramolecular polymer', 'host-guest complex', 'ammonothermal GaN:Mn', 'gallium compounds', 'photoluminescence decays', 'pump&#x2013;probe decays', 'time-resolved photo-carrier transport', 'wide-bandgap semiconductors', '7,7&#x2032;,8,8&#x2032;-tetracyanoquinodimethane', 'electrochemical (bio)sensors', 'electrode materials', 'metal/covalent&#x2013;organic frameworks', 'radical anion salts', 'sustainable batteries', 'voltammetry', 'bicarbazole-based donor', 'density functional theory', 'dopant-free', 'hole transporting material', 'perovskite solar cell', 'ATP regeneration', 'Burkholderia cenocepacia', 'biocatalysis', 'cascade reactions', 'cofactor recycling', 'polyphosphate', 'polyphosphate kinase', 'D-A copolymer', 'flexible electronic', 'ion-gel dielectric', 'organic field effect transistors', 'organic semiconductor', 'ROS'

...

Extracted Keywords: ['Active surveillance', 'Genomic testing', 'Gleason upgrade', 'PI-RADS 5', 'Prostate MRI', 'Prostate biopsy', 'Prostate cancer', 'arterial insufficiency', 'mesenchymal stem cell', 'mitochondria', 'nicotine', 'smoking', 'Cellular senescence &amp; aging', 'Chemical warfare-associated agents', 'Microvesicle particles', 'Platelet-activating factor', 'Signaling pathways', 'Sulfur and nitrogen mustards', 'Anticoagulants', 'Atrial fibrillation', 'Clinical coding', 'Hemorrhage', 'Intracranial hemorrhages', 'Collagens', 'LOX', 'Mustard gas keratopathy', 'Myofibroblast', 'Sulfur mustard', 'cornea', 'acute cerebral ischemia', 'acute reperfusion treatment', 'ischemic stroke', 'microembolic signals', 'secondary stroke prevention', 'stroke subtypes', 'transcranial doppler', 'Glomerular Endothelial Cells', 'KLF4', 'PLVAP', 'Thrombotic Microangiopathy', 'Deep brain stimulation', 'Parkinson&#x2019;s disease', 'Polysomnography', 'Sleep quality', 'Sleep spindle', 'Subthalamic nucleus'

Extracted Keywords: ['intensive care', 'neonatal intensive care unit', 'pandemics', 'parent', 'severe acute respiratory syndrome coronavirus 2', 'Clock Drawing Test', "Parkinson's disease", 'dementia', 'HAI', 'bacteria', 'fomite-borne risk pathogens', 'fomites', 'fungi', 'inanimate surfaces', 'persistence', 'protozoa', 'replication capacity', 'resilience', 'tenacity', 'transmission', 'viability', 'viruses', 'And facilitators', 'Barriers', 'Healthcare workers', 'Infection prevention and control', 'Great Barrier Reef', 'Takotsubo cardiomyopathy', 'cardiac arrest', 'cardiology', 'coronary angiography', 'myocardial infarction', 'retrieval', 'troponin', 'Australia', 'Chemotherapy', 'exposure', 'healthcare workers', 'nurses', 'toilet aerosols', 'COVID&#x2010;19', 'intensive care', 'no visitor', 'visit restriction', 'artificial intelligence', 'hospital management', 'hospital-acquired infection', 'indoor air quality']
Article Links: ['https://pubmed.ncbi.nlm.nih.gov/39412373/', 'https://pubmed

Extracted Keywords: ['United Arab Emirates', 'cancer health literacy', 'cancer screening', 'health literacy', 'knowledge', 'Agent-based model', 'Healthcare-associated infection', 'IPC', 'Individual-based model', 'Infection prevention and control', 'Masking', 'COVID-19', 'Disruptions', 'Essential health services', 'In-person visits', 'PHC service delivery', 'PHC service utilization', 'Patient-provider communication', 'Primary healthcare', 'Telemedicine', 'Vulnerable populations', 'Parkinson disease', 'health campaign', 'management options', 'neurodegenerative diseases', 'public health awareness', 'symptoms perception', "Child's visits", 'Custodians', 'Experiences', 'Intensive care', 'Parents', 'communicable diseases', 'disease outbreaks', 'epidemics', 'health personnel', 'immunization', 'autism', 'foreign body', 'intellectual', 'laparotomy', 'obstruction']
Article Links: ['https://pubmed.ncbi.nlm.nih.gov/38717601/', 'https://pubmed.ncbi.nlm.nih.gov/38714946/', 'https://pubmed.ncbi.nlm.n

Processing concepts:  10%|█▊                 | 64/663 [30:54<4:49:17, 28.98s/it]


ConnectionError: ('Connection aborted.', OSError(65, 'No route to host'))

In [ ]:
import csv
from collections import Counter
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Step 1: Load Keywords from CSV and Process Them
def load_and_process_keywords(input_csv):
    keywords_counter = Counter()
    
    with open(input_csv, mode='r') as infile:
        reader = csv.DictReader(infile)
        
        for row in reader:
            keywords_str = row.get('Top_Keywords_and_Counts', '')
            
            # Extract keywords and their counts from the formatted string
            # The regex captures multi-word phrases, including those with hyphens and numbers
            keywords_with_counts = re.findall(r'([\w\s\-]+): (\d+)', keywords_str)
            
            # Update the counter with lowercase keywords and their respective counts
            for keyword, count in keywords_with_counts:
                keywords_counter[keyword.strip().lower()] += int(count)
    
    return keywords_counter

# Step 2: Find Common Keywords
def find_common_keywords(keywords_counter, min_count=1):
    # Filter keywords that appear across many concepts based on the min_count threshold
    common_keywords = {keyword: count for keyword, count in keywords_counter.items() if count >= min_count}
    
    # Sort keywords by count (descending order)
    sorted_keywords = sorted(common_keywords.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_keywords

# Step 3: Suggest New Semantic Types Based on Common Keywords
def suggest_semantic_types(sorted_keywords, top_n=10):
    print(f"Suggesting top {top_n} semantic types based on the most common keywords:\n")
    for keyword, count in sorted_keywords[:top_n]:
        print(f"Keyword: {keyword}, Count: {count} (Potential Semantic Type)")

# Step 4: Create a Word Cloud from the Top Keywords
def create_word_cloud(keywords_counter, top_n=100):
    # Get the top_n keywords
    top_keywords = keywords_counter.most_common(top_n)
    
    # Convert the top keywords into a dictionary suitable for WordCloud
    word_freq = {keyword: count for keyword, count in top_keywords}
    
    # Create the word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
    
    # Display the word cloud using matplotlib
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

# Main Function
def main():
    input_csv = '../data/semantic_network_umls_terms_final_with_keywords.csv'
    
    # Step 1: Load and process keywords from the CSV file
    keywords_counter = load_and_process_keywords(input_csv)
    
    # Step 2: Find common keywords
    sorted_keywords = find_common_keywords(keywords_counter, min_count=2)  # Adjust min_count as needed
    
    # Step 3: Suggest new semantic types based on common keywords
    suggest_semantic_types(sorted_keywords, top_n=50)  # Adjust top_n as needed
    
    # Step 4: Create a word cloud from the top 100 keywords
    create_word_cloud(keywords_counter, top_n=50)

if __name__ == "__main__":
    main()